### Imports

In [1]:
import warnings 
warnings.filterwarnings('ignore')

import torch
import pprint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import SeamlessM4Tv2Model, AutoProcessor

from src.tokenize import AggregatedTokenizer
from src.generate import EnsembleGenerator
from src.evaluate import SimilarityChecker


### Config

In [2]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

### Initialize single translator-models for ensemble 

In [3]:
# !padding_side affects translation quality greatly!

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-3.3B", padding_side="left")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-3.3B").to(device)

processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large", padding_side="left")
model2 = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large").to(device)

tokenizer3 = AutoTokenizer.from_pretrained("google/madlad400-3b-mt", padding_side="left")
model3 = AutoModelForSeq2SeqLM.from_pretrained("google/madlad400-3b-mt").to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Initialize similarity checker

In [4]:
similarity_checker = SimilarityChecker(score_names=['bertscore', 'sentence_chrf'])

### Initialize aggregated tokenizer

In [5]:
agg_tokenizer = AggregatedTokenizer(
    tokenizers=[
        tokenizer, 
        processor.tokenizer,
        tokenizer3,
    ],
    tokenization_kwargs=[
        dict(),
        dict(src_lang="eng", tgt_lang="rus"),
        dict(),
    ],
    decoder_tokenization_postprocessing=[
        None,
        None, 
        None
    ]
)

Adding tokens from tokenizer 0 to aggregated tokenizer --> Done
Adding tokens from tokenizer 1 to aggregated tokenizer --> Done
Adding tokens from tokenizer 2 to aggregated tokenizer --> Done


### Initialize ensemble generator 

In [6]:
ensemble_generator = EnsembleGenerator(
    models=[
        model, 
        model2,
        model3,
    ],
    generation_kwargs=[
        dict(),
        dict(generate_speech=False),
        dict(),

    ],
    agg_tokenizer=agg_tokenizer,
    similarity_checker=similarity_checker,
    decoder_prompts = [
        "</s> rus_Cyrl",
        "</s> __rus__",
        "<unk>",
    ],
    encoder_prompts = [
        None,
        None,
        "<2ru>"
    ]
)

### Ensemble text translation example

In [9]:
%%time
eng_text = """
Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, 
is an Australian-based airline. It is the largest airline by fleet size 
to use the Virgin brand. It commenced services on 31 August 2000 as 
Virgin Blue, with two aircraft on a single route. It suddenly found 
itself as a major airline in Australia's domestic market after the 
collapse of Ansett Australia in September 2001. The airline has since 
grown to directly serve 32 cities in Australia, from hubs in Brisbane, 
Melbourne and Sydney."
"""

"""
 For text above:
-----------------------------------------------------------------------------------
 - instance translations generation: ~10s (num_beams=5)
 - ensemble translation generation:  ~1m 30s (num_beams=3) | ~1m 10s (num_beams=2) 
 - translations evaluation:          ~2s
-----------------------------------------------------------------------------------
  Total:                             ~1m 40s

"""

response_translated = ensemble_generator.translate(
    eng_text, 
    device=device,
    ensemble_num_beams=5,
    instance_num_beams=5,
    max_new_tokens=256,
    verbose=False
)
response_translated

CPU times: user 1min 41s, sys: 3.82 s, total: 1min 45s
Wall time: 1min 52s


{'ensemble_translation': 'Virgin Australia, торговое название Virgin Australia Airlines Pty Ltd, является австралийской авиакомпанией. Это крупнейшая авиакомпания по размеру флота, использующая бренд Virgin. Она начала свои услуги 31 августа 2000 года как Virgin Blue, с двумя самолетами на одном маршруте. Она внезапно оказалась крупной авиакомпанией на внутреннем рынке Австралии после краха Ansett Australia в сентябре 2001 года. С тех пор авиакомпания выросла, чтобы напрямую обслуживать 32 города в Австралии из узлов в Брисбене, Мельбурне и Сиднее.',
 'instance_translations': [' Virgin Australia, торговое название Virgin Australia Airlines Pty Ltd, является австралийской авиакомпанией. Это крупнейшая авиакомпания по размеру флота, использующая бренд Virgin. Она начала обслуживание 31 августа 2000 года как Virgin Blue, с двумя самолетами на одном маршруте. Она внезапно оказалась крупной авиакомпанией на внутреннем рынке Австралии после краха Ansett Australia в сентябре 2001 года. Авиако